In [30]:
import pandas as pd

In [7]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [51]:
df = pd.read_csv('../datalake/bronze/events.csv')

df[['Discipline', 'non_medal']] = df['Discipline'].str.extract(r'^(.*)\s*\(([^()]*)\)$', expand=True)
df[['Event', 'note']] = df['Event'].str.extract(r'^(.*)\s*\(([^()]*)\)$', expand=True)

In [52]:
df.head()

,Discipline,Event,non_medal,note
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [47]:
df.iloc[46:58]

,Discipline,Event,non_medal,discontinued
46,NaN,"Unknown Event, Men",NaN,discontinued
47,NaN,"Stadion, Men",NaN,discontinued
48,NaN,"Diaulos, Men",NaN,discontinued
49,NaN,Race in Armor,NaN,"Hoplite Race), Men (discontinued"
50,NaN,"Dolikhos, Men",NaN,discontinued
51,NaN,"Foot Race, Men",NaN,discontinued
52,NaN,"Pentathlon, Men",NaN,discontinued
53,NaN,"Special Event, Men",NaN,discontinued
54,NaN,"Stadion / Diaulos, Men",NaN,discontinued
55,NaN,"Diaulos or Race in Armour, Men",NaN,discontinued


In [44]:
df[df['Discipline'] == 'Ancient Athletics']

,Discipline,Event,non_medal,discontinued


In [39]:
df['discontinued'].unique()

array([nan, 'discontinued',
       'Wind Instrument) Competition, Men  (discontinued',
       'Hoplite Race), Men  (discontinued', 'Keles), Men  (discontinued',
       'Kalpe), Men  (discontinued',
       'Mule-Cart Race), Men  (discontinued', '#2), Men  (discontinued',
       'Dolikhos), Men  (discontinued', 'Ancient), Men  (discontinued',
       '<250 kg), Men  (discontinued', '<400 kg), Men  (discontinued',
       '#1), Handicap, Open  (discontinued',
       '#2), Handicap, Open  (discontinued', '#3), Open  (discontinued',
       '#4), Second Degree, Open  (discontinued',
       '#1), Open  (discontinued', '#2), Open  (discontinued',
       '#3), Combined, Open  (discontinued',
       '#3), Intermediate stop, Open  (discontinued',
       '#3), Final stop, Open  (discontinued',
       '#4), Self-selected target, Open  (discontinued',
       'large), Open  (discontinued', 'medium), Open  (discontinued',
       'small), Open  (discontinued', 'Open), Open  (discontinued',
       'Under 

In [9]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO

events_url = 'https://www.olympedia.org/event_names'
response = requests.get(events_url, timeout=60)

soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('table', {'class': 'table'})

In [12]:
table

<table class="table">
<tr class="even">
<td>
<h2>
3-on-3 Ice Hockey
</h2>
</td>
</tr>
<tr class="odd">
<td>
<a href="/event_names/2315">3-on-3 Ice Hockey, Boys</a>
</td>
</tr>
<tr class="even">
<td>
<a href="/event_names/2316">3-on-3 Ice Hockey, Girls</a>
</td>
</tr>
<tr class="even">
<td>
<h2>
3x3 Basketball
</h2>
</td>
</tr>
<tr class="odd">
<td>
<a href="/event_names/2543">3x3 Basketball, Men</a>
</td>
</tr>
<tr class="even">
<td>
<a href="/event_names/2544">3x3 Basketball, Women</a>
</td>
</tr>
<tr class="odd">
<td>
<a href="/event_names/2202">3x3 Basketball, Boys</a>
</td>
</tr>
<tr class="even">
<td>
<a href="/event_names/2285">Dunk Contest, Boys</a>
</td>
</tr>
<tr class="odd">
<td>
<a href="/event_names/2203">3x3 Basketball, Girls</a>
</td>
</tr>
<tr class="even">
<td>
<a href="/event_names/2286">Shoot-out Contest, Girls</a>
</td>
</tr>
<tr class="even">
<td>
<h2>
Acrobatic Gymnastics
</h2>
</td>
</tr>
<tr class="odd">
<td>
<a href="/event_names/2462">Pairs, Mixed Youth</a>
</t

In [28]:
discipline = []
events = []

for row in table.find_all('tr'):
    discipline_tag = row.find('h2')
    if discipline_tag:
        current_discipline = discipline_tag.text.strip().replace('\n', ' ')
    else:
        event_tag = row.find('td')
        if event_tag and current_discipline:
            event_name = event_tag.text.strip().replace('\n', ' ')
            discipline.append(current_discipline)
            events.append(event_name)

events = pd.DataFrame({
    'Discipline': discipline,
    'Event': events
})

In [29]:
events[events['Discipline'] == 'Firefighting (non-medal only)']

,Discipline,Event
855,Firefighting (non-medal only),"Maneuvering, National, Men (discontinued)"
856,Firefighting (non-medal only),"Maneuvering, International, Volunteer Firefighters, Men (discontinued)"
857,Firefighting (non-medal only),"Maneuvering, International, Professional Firefighters, Men (discontinued)"
858,Firefighting (non-medal only),"Maneuvering, International, British Firefighters, Men (discontinued)"
